PROJECTS

[algospot의 PROJECTS 링크](https://algospot.com/judge/problem/read/PROJECTS)  
  
### 문제  
돈이 되는 사업이라면 무엇이든지 하는 음흉한 다국적기업 레이너 유한회사는 이번에 국책 사업 몇 개에 응찰하려고 합니다. 국책 사업은 수요도 안정적이고, 지불도 확실하기 때문에 레이너 유한회사의 매출에 크게 기여 할 것으로 전망되고 있습니다.  
그러나 모든 국책 사업이 돈이 되는 것은 아닙니다. 각 사업을 위해서는 필요한 장비가 있고 해당 장비를 구매하기 위해서는 일정한 돈이 들어가기 때문입니다.  
  
국책 사업  

| 사업명 | 예상 수익 | 필요 장비 |
|:---:|:---:|:---:|
| 레이스 생산 | 260 | 스타포트 |
| 발키리 생산 | 60 | 스타포트, 컨트롤 타워, 아머리 |
| 골리앗 생산 | 140 | 팩토리, 아머리 |
| 벌처 생산 | 350 | 팩토리 |
| 시즈 탱크 생산 | 500 | 팩토리, 머신샵 |
  
장비  

| 장비명 | 구매 가격 |
|:---:|:---:|
| 스타포트 | 250 |
| 컨트롤 타워 | 100 |
| 아머리 | 150 |
| 팩토리 | 300 |
| 머신샵 | 100 |
  
앞의 표는 레이너 유한회사가 응찰하려고 하는 국책 사업들과 예상되는 수익, 각 사업에 필요한 장비 목록과 각 장비의 가격을 보여줍니다. 한 장비가 두 개 이상의 국책 사업에 필요할 경우에는 하나만 구입해도 됩니다. 이 때 순이익은 국책 사업의 예상 수입에서 장비 구매 가격을 제한 것입니다.  
이 때 레이너 유한회사가 얻을 수 있는 최대 순이익은 얼마일까요?  
응찰한 국책 사업에는 반드시 낙찰된다고 가정합니다.  

### 출력  
한 줄에 레이너 유한회사가 얻을 수 있는 최대 순이익을 출력합니다.  

solution의  
expected_revenues : 각 국책 사업당 예상 수익 $(1 \leq len(expected\_revenues) \leq 100)$  
costs : 각 장비의 구매 비용 $(1 \leq len(costs) \leq 100)$  
equipment_need : [각 국책 사업별 장비의 필요여부]를 가지는 리스트  
0: 장비 필요 X  
1: 장비 필요 O  


In [1]:
import sys
from queue import Queue

def networkFlow(source, sink, V, capacity):
# 네트워크를 따라 source에서 sink로 보낼 수 있는 최대 유량 반환
    flow = [[0 for _ in range(V)] for _ in range(V)]
    
    totalFlow = 0 # 최대 유량
    while True: 
        parent = [-1 for _ in range(V)] # parent[u] : u번 노드의 parent
        q = Queue() # 너비 우선 탐색
        parent[source] = source; # 출발점의 부모는 자기 자신
        q.put(source) 
        while (not q.empty()) and parent[sink] == -1:
        # 큐가 비지 않고, 새로 탐색하는 노드에 대해서
            here = q.get() # 현재 노드
            for there in range(V): # 모든 노드에 대해서
                if capacity[here][there] - flow[here][there] > 0 and parent[there] == -1:
                # 잔여 용량이 있고, 현재노드의 자식인 노드에 대해서
                    q.put(there) # 다시 큐에 넣어서 탐색
                    parent[there] = here # 해당 노드의 부모는 지금 현재 탐색하던 노드
        if parent[sink] == -1: break # 증가하는 경로가 없다면 종료
        
        amount = sys.maxsize 
        p = sink # 도착지 노드부터 시작해서 다시 탐색
        while p != source: 
            amount = min(amount, capacity[parent[p]][p] - flow[parent[p]][p])
            p = parent[p]
        # 증가경로에서 얼마나 많은 유량을 보낼 수 있는지 탐색
        
        p = sink # 도착지 노드부터 시작해서 탐색
        while p != source:
            flow[parent[p]][p] += amount
            flow[p][parent[p]] -= amount
            p = parent[p]
        # 증가경로에서 보낸 유량 만큼 흐르는 유량을 갱신해줌.
        totalFlow += amount # 보내는 유량을 더함
    return totalFlow # 답 반환

def solution(expected_revenues, costs, equipment_need):
    source = 0 # 소스
    sink = 1 # 싱크
    n, m = len(expected_revenues), len(costs) # 사업 개수, 장비 개수
    V = n + m + 2 # 사업 개수 + 장비 개수 + (소스, 싱크)
    capacity = [[0 for _ in range(V)] for _ in range(V)] # 수용용량
    for i in range(n):
        capacity[source][2+i] = expected_revenues[i]
        # 소스부터 사업까지의 용량은 예상 수익으로 설정
    for i in range(m):
        capacity[2+n+i][sink] = costs[i]
        # 장비부터 싱크까지의 용량은 해당 장비의 구매 가격으로 설정
    for i in range(n):
        for j in range(m):
            if equipment_need[i][j] == 1: # i번 사업에서 j번째 장비를 구매해야할 때
                capacity[2+i][2+n+j] = sys.maxsize
                # 해당하는 사업에서 장비까지의 용량을 무한대로 잡는다.
    M = sum(expected_revenues) # 모든 사업의 예상 수익의 합
    C = networkFlow(source, sink, V, capacity)
    # 최소 컷의 용량 == 네트워크의 최대 유량 이므로
    # 네트워크의 최대 유량을 찾는다.
    return M - C # 답 반환

In [2]:
expected_revenues = [10, 10]
costs = [5, 10]
equipment_need = [[1, 0],[1, 1]]

In [3]:
solution(expected_revenues, costs, equipment_need)

5

In [4]:
expected_revenues = [260, 60, 140, 350, 500]
costs = [250, 100, 150, 300, 100]
equipment_need = [[1, 0, 0, 0, 0],[1, 1, 1, 0, 0],[0, 0, 1, 1, 0],[0, 0, 0, 1, 0],[0, 0, 0, 1, 1]]

In [5]:
solution(expected_revenues, costs, equipment_need)

460